# CAPSTON PROJECT

---



## STEP 1: LOAD and IMPORT LIBRARIES

---



In [1]:
import os
import re
import string
import pandas as pd
import numpy as np

## STEP 2: IMPORT DATA

---



In [2]:
#dataset_df = pd.read_csv("Resources/fake_job_postings.csv")
dataset_df = pd.read_csv("https://raw.githubusercontent.com/Ikyupark/Capstone-project/main/Resources/fake_job_postings.csv")
dataset_df.head(1)

#import psycopg2
#import getpass
#db_password = getpass.getpass(prompt='DB Password ')

#engine = psycopg2.connect(
#    database="Capstone_Project_Group_9",
#    user="postgres",
#    password=db_password,
#    host="capstone-project-group-9.c4857qrbkptn.us-east-2.rds.amazonaws.com",
#    port='5432'
#)

#dataset_df = pd.read_sql('SELECT * FROM Test', engine)
#dataset_df.head(5)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,US; NY; New York,Marketing,NaN,We're Food52 and we've created a groundbreakin...,Food52 a fast-growing James Beard Award-winnin...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0


In [3]:
# Drop columns with too many null values or where all values are unique
dataset_df = dataset_df.drop(columns=['job_id','salary_range','title'])

In [4]:
# Display number of null values for each column
dataset_df.isnull().sum()

location                 346
department             11547
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [5]:
# Split the location columns by the seperator and only keep the country, state and city
dataset_df['Country'] = dataset_df['location'].str.split(';').str[0]
dataset_df = dataset_df.drop(columns=['location'])
dataset_df.head(1)

,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country
0,Marketing,We're Food52 and we've created a groundbreakin...,Food52 a fast-growing James Beard Award-winnin...,Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US


In [6]:
# Process to remove all characters from the data
clean_cols = ["company_profile", "description", "requirements", "benefits"]
punctuation = string.punctuation+r'[^a-zA-Z0-9\s]'

In [7]:
# Replace null values with empty string
##dataset_df = dataset_df.fillna('')
##dataset_df.head(1)

for col in clean_cols:
  dataset_df[col] = dataset_df[col].replace(np.nan,'')

In [8]:
# Define function that will remove punctuation from text
def clean_punctuations(text):
    for char in punctuation:
      if text !=np.nan:
        text = text.replace(char, '')
    return text

In [9]:
# Loop through clean_cols and remove punctuation and characters
for col in clean_cols:
    dataset_df[col] = dataset_df[col].apply(clean_punctuations)
    #dataset_df[col] = dataset_df[col].replace(r'[^a-zA-Z0-9\s]', '',regex=True)
    #dataset_df[col] = dataset_df[col].replace(r'\s{2,}', '',regex=True)

dataset_df.head(1)

,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,Country
0,Marketing,Were Food52 nd weve creted groundbreking nd w...,Food52 ftgrowing Jme Berd wrdwinning online f...,Experience with content mngement ytem mjor pl...,,0,1,0,Other,Internship,NaN,NaN,Marketing,0,US


In [10]:
# check the number of null values in columns
dataset_df.isnull().sum()

department             11547
company_profile            0
description                0
requirements               0
benefits                   0
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
Country                  346
dtype: int64

In [11]:
# Remove any whitespace from the country, city and state columns
### No Longer Required ###
#dataset_df['Country'] = dataset_df['Country'].map(lambda x: x.strip())

#dataset_df.head()

In [12]:
# make all text lower case
string_cols = list(dataset_df.select_dtypes(include='object'))
for col in string_cols:
    dataset_df[col] = dataset_df[col].str.lower()

#dataset_df.head()

In [13]:
# Temporary code block to write cleaned data to csv at this point
#dataset_df.to_csv('/content/drive/MyDrive/Colab Notebooks/fake_job_postings_cleaned.csv')

In [14]:
# Get number of unique values in each column
dataset_df.nunique()

department              1283
company_profile         1710
description            14762
requirements           11924
benefits                6104
telecommuting              2
has_company_logo           2
has_questions              2
employment_type            5
required_experience        7
required_education        13
industry                 131
function                  37
fraudulent                 2
Country                   90
dtype: int64

## Step 3: Value Bucketing

---



In [15]:
# Define function that will keep the top n number of values in a column and change the rest to other
def top_n_values(df_column, n):
  df_column_counts = df_column.value_counts()
  replace_df_column = list(df_column_counts[n:].index)

  # Replace in DataFrame
  for val in replace_df_column:
    df_column = df_column.replace(val,"Other")

  return df_column

In [16]:
dataset_df['department'] = top_n_values(dataset_df['department'], 10)
dataset_df.department.value_counts()

Other                     3853
sales                      568
engineering                494
marketing                  403
operations                 271
it                         226
development                146
product                    114
information technology      87
customer service            86
tech                        85
Name: department, dtype: int64

In [17]:
dataset_df['industry'] = top_n_values(dataset_df['industry'], 10)
dataset_df.industry.value_counts()

Other                                  4892
information technology and services    1734
computer software                      1376
internet                               1062
marketing and advertising               828
education management                    822
financial services                      779
hospital & health care                  497
consumer services                       358
telecommunications                      342
oil & energy                            287
Name: industry, dtype: int64

In [18]:
dataset_df['function'] = top_n_values(dataset_df['function'], 10)
dataset_df.function.value_counts()

Other                     2843
information technology    1749
sales                     1468
engineering               1348
customer service          1229
marketing                  830
administrative             630
design                     340
health care provider       338
other                      325
education                  325
Name: function, dtype: int64

In [19]:
dataset_df['Country'] = top_n_values(dataset_df['Country'], 10)
dataset_df.Country.value_counts()

us       10656
gb        2384
Other     1632
gr         940
ca         457
de         383
nz         333
in         276
au         214
ph         132
nl         127
Name: Country, dtype: int64

In [20]:
## required_education
# combine education values that are not formal education
required_education_vals = ['Vocational', 'Vocational - HS Diploma', 'Vocational - Degree', 'Professional', 'Some High School Coursework', 'Some College Coursework Completed']
dataset_df.loc[(dataset_df['required_education'].isin(required_education_vals), 'required_education')] = 'other'

dataset_df.required_education.value_counts()

bachelor's degree                    5145
high school or equivalent            2080
unspecified                          1397
master's degree                       416
associate degree                      274
certification                         170
some college coursework completed     102
professional                           74
vocational                             49
some high school coursework            27
doctorate                              26
vocational - hs diploma                 9
vocational - degree                     6
Name: required_education, dtype: int64

In [21]:
##Employment_Type
# Determine which values to replace
counts = dataset_df.employment_type.value_counts()
replace_employment_type = list(counts[counts < 1000].index)

# Replace in DataFrame
for val in replace_employment_type:
    dataset_df.employment_type = dataset_df.employment_type.replace(val,"Other")


# Check to make sure binning was successful
dataset_df.employment_type.value_counts()

full-time    11620
contract      1524
Other         1265
Name: employment_type, dtype: int64

In [22]:
dataset_df.dtypes

department             object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
Country                object
dtype: object

## Step 4: Tockenizing

---



### NLTK 

In [25]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# create stopwords
stop = stopwords.words('english')

In [27]:
# for each column remove stopwords
for col in clean_cols:
  dataset_df[col] = dataset_df[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [28]:
porter = PorterStemmer()

for col in clean_cols:
  dataset_df[col] = dataset_df[col].apply(porter.stem)

In [29]:
# get_dummies on catageorical columns
dummy_col = ['department', 'employment_type', 'required_experience', 'required_education' , 'industry', 'function', 'Country']
for col in dummy_col:
    # Get one hot encoding of columns
    one_hot = pd.get_dummies(dataset_df[col],prefix=col,dtype=np.int64)
    # Drop column as it is now encoded
    dataset_df = dataset_df.drop(col,axis = 1)
    # join the encoded df with dataset_df
    dataset_df = dataset_df.join(one_hot)

In [30]:
dataset_df.head(2)

,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,fraudulent,department_Other,department_customer service,...,Country_au,Country_ca,Country_de,Country_gb,Country_gr,Country_in,Country_nl,Country_nz,Country_ph,Country_us
0,food52 nd weve creted groundbreking nd wrdwinn...,food52 ftgrowing jme berd wrdwinning online fo...,experience content mngement ytem mjor plu ny b...,,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,second world cloud video production service se...,orgnied focued vibrnt weomedo hve pion cutomer...,wht expect youyour key reponibility communicte...,wht get uthrough prt second tem ginexperience ...,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [31]:
# Create list of columns of type int64
num_cols = list(dataset_df.select_dtypes(include='int64'))
    
# Join all columns in clean_cols to have a single colums to vectorize
dataset_df['text'] = dataset_df['company_profile'] + dataset_df['description'] + dataset_df['requirements'] + dataset_df['benefits']
num_cols.append('text')

### View Resulting Output from NLTK

In [32]:
dataset_df.head(2)

,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,fraudulent,department_Other,department_customer service,...,Country_ca,Country_de,Country_gb,Country_gr,Country_in,Country_nl,Country_nz,Country_ph,Country_us,text
0,food52 nd weve creted groundbreking nd wrdwinn...,food52 ftgrowing jme berd wrdwinning online fo...,experience content mngement ytem mjor plu ny b...,,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,food52 nd weve creted groundbreking nd wrdwinn...
1,second world cloud video production service se...,orgnied focued vibrnt weomedo hve pion cutomer...,wht expect youyour key reponibility communicte...,wht get uthrough prt second tem ginexperience ...,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,second world cloud video production service se...


In [33]:
# Create dataframe with only required columns for vectorization
vector_df = dataset_df[num_cols]

In [34]:
vector_df.head(1)

,telecommuting,has_company_logo,has_questions,fraudulent,department_Other,department_customer service,department_development,department_engineering,department_information technology,department_it,...,Country_ca,Country_de,Country_gb,Country_gr,Country_in,Country_nl,Country_nz,Country_ph,Country_us,text
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,food52 nd weve creted groundbreking nd wrdwinn...


In [35]:
# Create Term freqeuncy Vectorizer
tfidf_vect = TfidfVectorizer(use_idf=True,
                      smooth_idf=True,
                      min_df = 100,
                      stop_words = 'english',
                      max_features = 10)

In [36]:
term_freq_idf_df = pd.DataFrame(tfidf_vect.fit_transform(vector_df['text']).toarray(), columns=tfidf_vect.get_feature_names_out())

In [37]:
term_freq_idf_df.head(2)

,buine,compny,cutomer,ervice,experience,hve,nd,tem,tht,work
0,0.046685,0.043202,0.000000,0.000000,0.000000,0.075183,0.990156,0.071530,0.00000,0.069114
1,0.150401,0.000000,0.153133,0.108674,0.156011,0.121106,0.908237,0.172831,0.20341,0.083497


### Create DF ready for Machine Learning

In [38]:
ml_df = vector_df.merge(term_freq_idf_df, left_index=True, right_index=True, how='inner')

In [39]:
ml_df.drop('text',axis=1,inplace=True)

In [40]:
ml_df.head(2)

,telecommuting,has_company_logo,has_questions,fraudulent,department_Other,department_customer service,department_development,department_engineering,department_information technology,department_it,...,buine,compny,cutomer,ervice,experience,hve,nd,tem,tht,work
0,0,1,0,0,0,0,0,0,0,0,...,0.046685,0.043202,0.000000,0.000000,0.000000,0.075183,0.990156,0.071530,0.00000,0.069114
1,0,1,0,0,1,0,0,0,0,0,...,0.150401,0.000000,0.153133,0.108674,0.156011,0.121106,0.908237,0.172831,0.20341,0.083497


## Step 5: Machine Learning
ml_df is the dataframe ready for machine learning
---